### Deploying custom docker model (purely inference)

In [ ]:
import sys
!{sys.executable} -m pip install --quiet sagemaker==1.72 -U

In [ ]:
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re


import sagemaker
from sagemaker import get_execution_role

In [ ]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [ ]:
docker_image_name = '<Name of docker image in registry on AWS>'

In [ ]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'anomaly-detection'

### Create Estimator with custom model

In [ ]:
sess = sagemaker.session.Session()
anomaly_detection = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    base_job_name="anomaly-detection",
                                    sagemaker_session=sess)

### Have to "train" model even if just for inference

In [ ]:
anomaly_detection.fit()

### Deploy custom model

In [ ]:
endpoint_name = 'anomaly-detection-endpoint'

In [ ]:
anomaly_detection.deploy(initial_instance_count=1, 
                                             instance_type='ml.t2.medium', 
                                             endpoint_name=endpoint_name)

In [ ]:
# Delete endpoint when you don't need your model deployed
sm.delete_endpoint(EndpointName=endpoint_name)